In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Colab\ Notebooks


/content/drive/MyDrive/Colab Notebooks


In [ ]:
!wget https://s3.amazonaws.com/datasets.huggingface.co/summarization/xsum.tar.gz

--2023-04-25 18:11:14--  https://s3.amazonaws.com/datasets.huggingface.co/summarization/xsum.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.97.78, 54.231.170.224, 52.217.129.72, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.97.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 204844092 (195M) [application/x-tar]
Saving to: ‘xsum.tar.gz.2’

xsum.tar.gz.2       100%[===================>] 195.35M  32.6MB/s    in 6.8s    

2023-04-25 18:11:21 (28.8 MB/s) - ‘xsum.tar.gz.2’ saved [204844092/204844092]



In [ ]:
!tar -xvzf xsum.tar.gz

xsum/
xsum/train.target
xsum/train.source
xsum/val.source
xsum/val.target
xsum/test.source
xsum/test.target


In [ ]:
cd /content/drive/MyDrive/Colab\ Notebooks/xsum

/content/drive/MyDrive/Colab Notebooks/xsum


In [ ]:
ls

test.source  test.target  train.source  train.target  val.source  val.target


In [ ]:
!pip install datasets  # Run this command if you haven't installed the datasets library
from datasets import load_dataset
dataset = load_dataset('xsum')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.6 MB/s eta 0:00:00


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        # get the data at the specified index
        example = self.data[idx]
        input_text = example['document']
        target_text = example['summary']
        # encode the input and target sequences using the tokenizer
        input_ids = self.tokenizer.encode(input_text, add_special_tokens=True, truncation=True, max_length=1024)
        target_ids = self.tokenizer.encode(target_text, add_special_tokens=True, truncation=True, max_length=128)
        # return the encoded input and target sequences as a dictionary
        return {'input_ids': input_ids, 'target_ids': target_ids}

    def __len__(self):
        # return the number of data points
        return len(self.data)

In [ ]:
print(f"Number of examples: {len(dataset['train'])}")
print(f"Available features: {dataset['train'].features}")

Number of examples: 204045
Available features: {'document': Value(dtype='string', id=None), 'summary': Value(dtype='string', id=None), 'id': Value(dtype='string', id=None)}


In [ ]:
example = dataset['train'][12]
print("Input text:\n", example['document'])
print('')
print("Summary:\n", example['summary'])

Input text:
 Administrators confirmed the redundancies affecting 38 staff at Galashiels-based Murray and Burrell.
The business, established in 1928, went into administration last week citing "adverse trading conditions".
There are hopes some of the workers affected could find posts at another building firm in nearby Melrose which currently requires staff.
Thomson Cooper partner Richard Gardiner was appointed as administrator at Murray and Burrell on Monday.
A statement confirmed: "Directors explored all options in an effort to preserve trading and jobs.
"Regrettably, 38 jobs were lost as there is no prospect of continuing to trade."
South of Scotland MSP Rachael Hamilton described it as a "sad day for the Borders".
However, some of the workers laid off could find employment with a Melrose-based company.
JS Crawford has said that, with several housing projects on its books, it needs staff.

Summary:
 Dozens of jobs have been lost after efforts to save an historic building firm in the Sc

In [ ]:
example = dataset['train'][204044]
print("Input text:\n", example['document'])
print('')
print("Summary:\n", example['summary'])

Input text:
 The incident occurred at the headquarters of the Siam Commercial Bank in Bangkok on Sunday night.
Contract workers were upgrading the fire extinguisher system when a chemical retardant was released, apparently suffocating them.
Police are investigating how the system was set off.
The executives, Adisorn Pokha and Napong Suksanguan, are from a firm called Mega Planet, which had contracted out the work.
They face up to 10 years in prison if found guilty of negligence.
Police Lt Gen Sanit Mahathaworn said they had expressed regret and were willing to pay compensation for damages, according to the Bangkok Post.
The bank had said that an aerosol system called pyrogen was "inadvertently activated and exhausted all the oxygen in the area" as contractors were working on it.
Pyrogen involves a mixture of gases including potassium carbonates, nitrogen, carbon dioxide, carbon monoxide, nitrogen oxide and ammonia. The system works by removing all oxygen from the air to stifle a fire.


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.5 MB/s eta 0:00:00


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F


# Load the BART tokenizer and model

In [ ]:
!pip install transformers

from transformers import BartTokenizer, BartForConditionalGeneration

# Load pre-trained BART model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-xsum')

# Load BART tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-xsum')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Define source
summarysource_text = "Chris Cox, the university's director of development, said the research centre would translate discoveries made in the laboratory into new treatments. He said it would house 150 additional researchers who will be developing new ideas and treatments. Research will focus on radiation therapy, lung cancer, women’s cancers, melanoma and haematological oncology. The centre is the result of a partnership between The University of Manchester, The Christie NHS Foundation Trust and Cancer Research UK. The Christie’s chief executive Caroline Shaw said the funding would 'help facilitate groundbreaking research right here in Manchester'."
print(summarysource_text)

Chris Cox, the university's director of development, said the research centre would translate discoveries made in the laboratory into new treatments. He said it would house 150 additional researchers who will be developing new ideas and treatments. Research will focus on radiation therapy, lung cancer, women’s cancers, melanoma and haematological oncology. The centre is the result of a partnership between The University of Manchester, The Christie NHS Foundation Trust and Cancer Research UK. The Christie’s chief executive Caroline Shaw said the funding would 'help facilitate groundbreaking research right here in Manchester'.


In [ ]:
!pip install transformers

from transformers import BartTokenizer, BartForConditionalGeneration

# Load pre-trained BART model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-xsum')

# Load BART tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-xsum')

# Define source and generate summary
source_text = "Chris Cox, the university's director of development, said the research centre would translate discoveries made in the laboratory into new treatments. He said it would house 150 additional researchers who will be developing new ideas and treatments. Research will focus on radiation therapy, lung cancer, women’s cancers, melanoma and haematological oncology. The centre is the result of a partnership between The University of Manchester, The Christie NHS Foundation Trust and Cancer Research UK. The Christie’s chief executive Caroline Shaw said the funding would 'help facilitate groundbreaking research right here in Manchester'."
inputs = tokenizer.encode(source_text, return_tensors='pt')
outputs = model.generate(inputs, max_length=60, min_length=10, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print generated summary
print(summary)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The University of Manchester is to open a £10m cancer research centre in Manchester city centre.


In [ ]:
# Print generated summary
print(summary)

The University of Manchester is to open a £10m cancer research centre in Manchester city centre.


In [ ]:
def generate_funding_value(source_text, model, tokenizer):
    # Encode the source text
    inputs = tokenizer.encode(source_text, return_tensors='pt')

    # Generate a funding value using MLE objective
    funding_logits = model(inputs).logits[0, -1, :]
    funding_prob = torch.softmax(funding_logits, dim=0)

    # Generate a random decimal value between 0 and 10 million
    funding_value = round(torch.rand(1).item() * 10000000, 2)

    # Print the generated funding value for debugging purposes
    print(f"Generated funding value: £{funding_value}m")

    return funding_value

In [ ]:
import random
summary_template = "The University of Manchester has been awarded £{:.1f}m to help fund a new cancer research centre in Manchester city centre."

num_summaries = 6
beam_size = 6

# Generate the summaries
for i in range(num_summaries):
    # Generate a random funding value using MLE objective
    #funding_value = generate_funding_value(source_text, model, tokenizer)

    # Create the input sequence with the generated funding value
    input_sequence = "The University of Manchester has been awarded £{:.1f}m to help fund a new cancer research centre in Manchester city centre."

    # Encode the input sequence with the tokenizer
    funding = random.uniform(0.5, 10.0)
    inputs = tokenizer.encode(input_sequence.format(funding), return_tensors='pt')

    # Generate the summary with BART
    outputs = model.generate(
        inputs,
        max_length=60,
        min_length=10,
        length_penalty=2.0,
        num_beams=beam_size,
        early_stopping=True,
        do_sample=True,
        top_p=0.9,
        temperature=0.8,
    )

    # Decode the summary with the tokenizer
    summary_template = "The University of Manchester has been awarded £{:.1f}m to help fund a new cancer research centre in Manchester city centre."
    for j, output in enumerate(outputs):
        summary = tokenizer.decode(output, skip_special_tokens=True)
        funding = random.uniform(0.5, 10.0)
        summary_text = summary_template.format(funding)
        print(f"Beam: {summary_text}")


Beam: The University of Manchester has been awarded £9.1m to help fund a new cancer research centre in Manchester city centre.
Beam: The University of Manchester has been awarded £5.7m to help fund a new cancer research centre in Manchester city centre.
Beam: The University of Manchester has been awarded £7.9m to help fund a new cancer research centre in Manchester city centre.
Beam: The University of Manchester has been awarded £0.6m to help fund a new cancer research centre in Manchester city centre.
Beam: The University of Manchester has been awarded £2.5m to help fund a new cancer research centre in Manchester city centre.
Beam: The University of Manchester has been awarded £0.5m to help fund a new cancer research centre in Manchester city centre.


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

input_sequence = "The University of Manchester has been awarded £{:.1f}m to help fund a new cancer research centre in Manchester city centre.".format(3.5)

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-xsum')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-xsum')

input_ids = tokenizer.encode(input_sequence, return_tensors='pt')




In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BartTokenizer, BartForConditionalGeneration

class AbstractiveSummarizationModel(nn.Module):
    def __init__(self, input_size, hidden_size, vocab_size):
        super(AbstractiveSummarizationModel, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.encoder = nn.LSTM(hidden_size, hidden_size, num_layers=1, batch_first=True)
        self.decoder = nn.LSTM(hidden_size, hidden_size, num_layers=1, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.fc_reject = nn.Linear(hidden_size, 2)
        self.m = 3
        self.alpha = 1e-3

    def forward(self, x, y):
        # Encoder
        embedded = self.embedding(x)
        hiddens, (last_hidden, last_cell) = self.encoder(embedded)

        # Decoder
        batch_size = x.size(0)
        decoder_input = torch.zeros((batch_size, 1)).long().to(x.device)
        h_t, c_t = last_hidden, last_cell
        outputs = []
        for t in range(y.size(1)):
            embedded = self.embedding(decoder_input)
            output, (h_t, c_t) = self.decoder(embedded, (h_t, c_t))
            output = self.fc(output.squeeze(1))

            # Rejection Loss
            pr_t = F.softmax(self.fc_reject(output), dim=-1)[:, -1]
            if t < self.m:
                loss = F.cross_entropy(output, y[:, t])
            else:
                loss = - (1 - pr_t) * F.cross_entropy(output, y[:, t], reduction='none')
                loss = torch.mean(loss)
                loss += self.alpha * torch.sum(-torch.log(1 - pr_t))

            outputs.append(output.unsqueeze(1))
            decoder_input = y[:, t].unsqueeze(1)

        return torch.cat(outputs, dim=1)

In [ ]:
# Encode the input sequence and generate the output sequence
input_ids = tokenizer.encode(input_sequence, return_tensors='pt')
outputs = model.generate(
    input_ids,
    max_length=60,
    num_beams=5,
    do_sample=False,
    use_cache=True,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    decoder_start_token_id=model.config.decoder_start_token_id,
    num_return_sequences=1,
)

# Decode the output sequence and replace the hallucinated token with <REJ>
output_sequence = tokenizer.decode(outputs[0], skip_special_tokens=True)
if '£3.5' in output_sequence:
    output_sequence = output_sequence.replace('£3.5', '£<REJ>')

# Print the output sequence
print(output_sequence)


The University of Manchester has been awarded £<REJ>m to help fund a new cancer research centre.


In [ ]:
!pip install datasets  # Run this command if you haven't installed the datasets library
from datasets import load_dataset
dataset = load_dataset('xsum')

xsum = load_dataset('xsum', split='train')
data = xsum['document']
summaries = xsum['summary']

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
batch_size = 8
num_epochs = 5
learning_rate = 3e-5
weight_decay = 0.01

In [ ]:
# Tokenize input and summary data
max_len_text = 500
max_len_summary = 50

tokenizer_text = Tokenizer()
tokenizer_text.fit_on_texts(data)
input_seq = tokenizer_text.texts_to_sequences(data)
input_seq = pad_sequences(input_seq, maxlen=max_len_text, padding='post')

tokenizer_summary = Tokenizer()
tokenizer_summary.fit_on_texts(summaries)
target_seq = tokenizer_summary.texts_to_sequences(summaries)
target_seq = pad_sequences(target_seq, maxlen=max_len_summary, padding='post')

In [ ]:
# Define the model
latent_dim = 256

encoder_inputs = Input(shape=(max_len_text,))
encoder_masking = Masking(mask_value=0.0)(encoder_inputs)
encoder_embedding = Embedding(input_dim=len(tokenizer_text.word_index)+1, output_dim=latent_dim, mask_zero=True)(encoder_masking)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_masking = Masking(mask_value=0.0)(decoder_inputs)
decoder_embedding = Embedding(input_dim=len(tokenizer_summary.word_index)+1, output_dim=latent_dim, mask_zero=True)(decoder_masking)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(len(tokenizer_summary.word_index)+1, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
# Define the loss function and compile the model
def masked_cross_entropy_loss(y_true, y_pred):
    mask = tf.math.logical_not(tf.math.equal(y_true, 0))
    mask = tf.cast(mask, dtype=tf.float32)
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=False)
    loss = tf.multiply(loss, mask)
    loss = tf.divide(tf.reduce_sum(loss), tf.reduce_sum(mask))
    return loss

model.compile(optimizer='adam', loss=masked_cross_entropy_loss, metrics=['accuracy'])

In [ ]:
# Compile the model
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with early stopping
#early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

history = model.fit([input_seq, target_seq[:,:-1]], target_seq[:,1:], epochs=5,validation_split=0.2) #callbacks=[early_stopping],

# Print training and validation accuracy and loss for each epoch
for epoch in range(len(history.history['loss'])):
    print(f"Epoch {epoch + 1}")
    print(f"Training loss: {history.history['loss'][epoch]}")
    print(f"Training accuracy: {history.history['accuracy'][epoch]}")
    print(f"Validation loss: {history.history['val_loss'][epoch]}")
    print(f"Validation accuracy: {history.history['val_accuracy'][epoch]}\n")


Epoch 1/5
5102/5102 [==============================] - 1454s 285ms/step - loss: 5.7691 - accuracy: 0.1754 - val_loss: 5.0859 - val_accuracy: 0.2244
Epoch 2/5
5102/5102 [==============================] - 1303s 255ms/step - loss: 4.6969 - accuracy: 0.2464 - val_loss: 4.6587 - val_accuracy: 0.2565
Epoch 3/5
5102/5102 [==============================] - 1287s 252ms/step - loss: 4.2246 - accuracy: 0.2778 - val_loss: 4.4968 - val_accuracy: 0.2710
Epoch 4/5
5102/5102 [==============================] - 1286s 252ms/step - loss: 3.9031 - accuracy: 0.3013 - val_loss: 4.4271 - val_accuracy: 0.2798
Epoch 5/5
5102/5102 [==============================] - ETA: 0s - loss: 3.6472 - accuracy: 0.3224